# Classify CIFAR

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### download data

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# 이미지를 보여주기 위한 함수
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# 학습용 이미지를 무작위로 가져오기
dataiter = iter(trainloader)
images, labels = dataiter.next()

# 이미지 보여주기
imshow(torchvision.utils.make_grid(images))
# 정답(label) 출력
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

### 그래프 생성


In [ ]:
import torch
from torch.optim import SGD
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
class CNN_classifier(torch.nn.Module):
    def __init__(self):
        super(CNN_classifier, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5) #inchannel, outchannel, kernel_size(5x5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120) # 앞서 나온 convolution 값 분류
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10) # 최종 분류 라벨 10개
        
    def forward(self, x):
        # functional 에는 nn과 비슷한 것들이 담겨있다. eg) relu . 하지만 사용법이 약간씩 다르다.
        xx = self.pool(F.relu(self.conv1(x)))
        print(xx.shape)
        xx = self.pool(F.relu(self.conv2(xx)))
        print(xx.shape)
        
        # (너비 * channel 이므로, 이 둘의 순서를 바꿔준다.)
        ## torch 의 tensor reshape => .view()
        xx = xx.view(-1, 16 * 5 * 5)
        print(xx.shape)
        
        xx = F.relu(self.fc1(xx))
        xx = F.relu(self.fc2(xx))
        xx = self.fc3(xx)
        
        return xx

CNN = CNN_classifier()
        

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
## optimizer는 학습될 파라미터들을 앞에 넣어줘야하는 거 잊지말자!
optimizer = SGD(CNN.parameters(), lr=1e-4, momentum=0.9)


In [ ]:
# epoch은 두 번만 돌리자
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        # 이걸 Variable로 감싸는 이유는 내 생각에 tensor로 변환하기 위함 같은디
        # 이제 Variable이 deprecated 되었으니 tensor로 해도 될듯
        inputs, labels = Variable(inputs), Variable(labels)
        
        outputs = CNN(inputs)
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0